# Identifying Wine Quality
This notebook hold a machine learning program to help identify wine qulity based on differnent attributes.
We will begin by importing the neccessary libriaries. Besides the normal libriaries we will use Scikit-Learn to split the data, clean the data and build a random forest. Next we will import packages to validate the models, and measure the model. Finally we will import ahte package to repeat the model for fture use

In [1]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.ensemble import RandomForestRegressor

from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV

from sklearn.metrics import mean_squared_error, r2_score
from sklearn.externals import joblib

C:\Users\buzzt\anaconda3\lib\site-packages\sklearn\externals\joblib\__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


Now that we have the neccesary packages we can move forward importing the data.  I have alread taken a look at the data and knoww we have a seperator of a semicolon which means we need to sseperat the data before mvoing forward.

In [2]:
dataset_url = 'http://mlr.cs.umass.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv'
data = pd.read_csv(dataset_url, sep=';')

Now that the data is in the program we can start to consider the basic information about the dataset. Lets start by looking at the column headers.

In [3]:
data.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


Looking at this table we can see that there are 12 different variables including quality.  Lets move on to the shape and size of the table.

In [4]:
data.shape

(1599, 12)

Ok this confirms that there are 12 columns and 1599 rows.  Lets consider the statistcs of the data.  

In [5]:
data.describe()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
count,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000
mean,8.319637,0.527821,0.270976,2.538806,0.087467,15.874922,46.467792,0.996747,3.311113,0.658149,10.422983,5.636023
std,1.741096,0.179060,0.194801,1.409928,0.047065,10.460157,32.895324,0.001887,0.154386,0.169507,1.065668,0.807569
min,4.600000,0.120000,0.000000,0.900000,0.012000,1.000000,6.000000,0.990070,2.740000,0.330000,8.400000,3.000000
25%,7.100000,0.390000,0.090000,1.900000,0.070000,7.000000,22.000000,0.995600,3.210000,0.550000,9.500000,5.000000
50%,7.900000,0.520000,0.260000,2.200000,0.079000,14.000000,38.000000,0.996750,3.310000,0.620000,10.200000,6.000000
75%,9.200000,0.640000,0.420000,2.600000,0.090000,21.000000,62.000000,0.997835,3.400000,0.730000,11.100000,6.000000
max,15.900000,1.580000,1.000000,15.500000,0.611000,72.000000,289.000000,1.003690,4.010000,2.000000,14.900000,8.000000


Looking at the count of each column we know that there are no missing data, unfortunatly we will need to stanrdize all of the data.  Let move on to spliting the data for the model.  We will start with making the target data to the y variable and all others to the x.  This will allow us to spit the data for training and testing in the future.

In [6]:
y = data.quality
X = data.drop('quality', axis=1)

Now that we have the two variables split out we can create the training and testing data sets.  for this model we will split 80% of the data into a training set and 20% for testing.  Wealso choose to split the data randomly based ont he seed value of 123.

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.2, 
                                                    random_state=123, 
                                                    stratify=y)

With the data split we need to move onto standardize the data. To do this we will use the standard scaler tool from SciKit-Learn.  Using this tool will allow for the same data transforation to be applied to the test set.  

In [8]:
pipeline = make_pipeline(preprocessing.StandardScaler(), 
                         RandomForestRegressor(n_estimators=100))

This Scaler variable now holds the mean and standard deviation for all of the features.  We will use this to transform our data sets. Next we need to consider the hyperparameters.  To consider these tunable features we will start by identifing them.

In [9]:
print(pipeline.get_params())

{'memory': None, 'steps': [('standardscaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('randomforestregressor', RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False))], 'verbose': False, 'standardscaler': StandardScaler(copy=True, with_mean=True, with_std=True), 'randomforestregressor': RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurit

Looking at this list of different paramaters we can see that there are many differnt parameters that we could consider but we really only want two this time. <br>randomforestregressor__max_depth<br>randomforestregressor__max_features<br>To accomplish this we will set the parameters up in a dictioanry with the different values that we want to pass.

In [10]:
hyperparameters = { 'randomforestregressor__max_features' : ['auto', 'sqrt', 'log2'],
                  'randomforestregressor__max_depth': [None, 5, 3, 1]}

Now that we have defined the hyperparameters we can move forward to setting up the cross validation.  To accomplish this we will use GridSearchCV which will create the eccesary folds.

In [11]:
clf = GridSearchCV(pipeline, hyperparameters, cv=10)
clf.fit(X_train, y_train)

GridSearchCV(cv=10, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('standardscaler',
                                        StandardScaler(copy=True,
                                                       with_mean=True,
                                                       with_std=True)),
                                       ('randomforestregressor',
                                        RandomForestRegressor(bootstrap=True,
                                                              ccp_alpha=0.0,
                                                              criterion='mse',
                                                              max_depth=None,
                                                              max_features='auto',
                                                              max_leaf_nodes=None,
                                                              max_samples=None,
                            

Now that we have the the training set complete we can review the parameters to see if we need to change anything.

In [12]:
print(clf.best_params_)

{'randomforestregressor__max_depth': None, 'randomforestregressor__max_features': 'sqrt'}


Based on this we will leave the default parameters and move forward to evaluating the model on the test set.  We will start by creating the prediction.

In [13]:
y_pred = clf.predict(X_test)

In [16]:
r2 =round(r2_score(y_test, y_pred), 2)
mse = round(mean_squared_error(y_test, y_pred), 2)
print('The R2 Score is', r2,'and the Mean Squared Error is', mse,'.')

The R2 Score is 0.46 and the Mean Squared Error is 0.35 .


The final step is to save the processs for futre test.  

In [17]:
joblib.dump(clf, 'rf_regressor.pkl')
#to load it use clf2 = joblib.load('rf_regressor.pkl')

['rf_regressor.pkl']